In [70]:
import pandas as pd
import numpy as np

In [71]:
pd.set_option('display.max_colwidth', 75)
df_2018 = pd.read_csv("2018_movies.csv")

In [72]:
df_2018.head()

,Title,cast_and_crew
0,Insidious The Last Key,"Adam Robitel (director); Leigh Whannell (screenplay); Lin Shaye, Angus ..."
1,The Strange Ones,Lauren Wolkstein (director); Christopher Radcliff (co-director/screenpl...
2,Stratton,"Simon West (director); Duncan Falconer, Warren Davis II (screenplay); D..."
3,Sweet Country,"Warwick Thornton (director); David Tranter, Steven McGregor (screenplay..."
4,The Commuter,"Jaume Collet-Serra (director); Byron Willinger, Philip de Blasi (screen..."


In [73]:
df_2018.tail()

,Title,cast_and_crew
1138,Sing 2,"Garth Jennings (director/screenplay); Matthew McConaughey, Reese Wither..."
1139,The King's Man,Matthew Vaughn (director/screenplay); Karl Gajdusek (screenplay); Ralph...
1140,The Tragedy of Macbeth,"Joel Coen (director/screenplay); Denzel Washington, Frances McDormand, ..."
1141,A Journal for Jordan,Denzel Washington (director); Virgil Williams (screenplay); Michael B. ...
1142,American Underdog,"Erwin brothers (directors); Jon Erwin, David Aaron Cohen, Jon Gunn (scr..."


In [74]:
!pip install tmdbv3api

In [75]:
from tmdbv3api import TMDb
import json
import requests
tmdb = TMDb()
tmdb.api_key = '9f61c4863cfdf70e7d30f998ae1f0f41'

In [76]:
from tmdbv3api import Movie
tmdb_movie = Movie()
def get_genre(x):
    genres = []
    result = tmdb_movie.search(x)
    if result == []:
        movie_id = np.NaN
    else:
        movie_id = result[0].id
        response = requests.get('https://api.themoviedb.org/3/movie/{}?api_key={}'.format(movie_id,tmdb.api_key))
        data_json = response.json()
        if data_json['genres']:
            genre_str = " " 
            for i in range(0,len(data_json['genres'])):
                genres.append(data_json['genres'][i]['name'])
            return genre_str.join(genres)
        else:
            np.NaN

In [77]:
df_2018['genres'] = df_2018['Title'].map(lambda x: get_genre(str(x)))

In [78]:
df_2018.head()

,Title,cast_and_crew,genres
0,Insidious The Last Key,"Adam Robitel (director); Leigh Whannell (screenplay); Lin Shaye, Angus ...",Horror Mystery Thriller
1,The Strange Ones,Lauren Wolkstein (director); Christopher Radcliff (co-director/screenpl...,Thriller Drama
2,Stratton,"Simon West (director); Duncan Falconer, Warren Davis II (screenplay); D...",Action Thriller
3,Sweet Country,"Warwick Thornton (director); David Tranter, Steven McGregor (screenplay...",Drama History Western
4,The Commuter,"Jaume Collet-Serra (director); Byron Willinger, Philip de Blasi (screen...",Action Thriller


In [79]:
def get_director(x):
    if " (director)" in x:
        return x.split(" (director)")[0]
    elif " (directors)" in x:
        return x.split(" (directors)")[0]
    else:
        return x.split(" (director/screenplay)")[0]

In [80]:
df_2018['director'] = df_2018['cast_and_crew'].map(lambda x: get_director(x))

In [81]:
def get_actor1(x):
    return ((x.split("screenplay); ")[-1]).split(", ")[0])

df_2018['actor1'] = df_2018['cast_and_crew'].map(lambda x: get_actor1(x))

In [82]:
def get_actor2(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 2:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[1])
    
df_2018['actor2'] = df_2018['cast_and_crew'].map(lambda x: get_actor2(x))

In [83]:
def get_actor3(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 3:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[2])

df_2018['actor3'] = df_2018['cast_and_crew'].map(lambda x: get_actor3(x))

In [84]:
df_2018.head()

,Title,cast_and_crew,genres,director,actor1,actor2,actor3
0,Insidious The Last Key,"Adam Robitel (director); Leigh Whannell (screenplay); Lin Shaye, Angus ...",Horror Mystery Thriller,Adam Robitel,Lin Shaye,Angus Sampson,Leigh Whannell
1,The Strange Ones,Lauren Wolkstein (director); Christopher Radcliff (co-director/screenpl...,Thriller Drama,Lauren Wolkstein,Alex Pettyfer,James Freedson-Jackson,Emily Althaus
2,Stratton,"Simon West (director); Duncan Falconer, Warren Davis II (screenplay); D...",Action Thriller,Simon West,Dominic Cooper,Austin Stowell,Gemma Chan
3,Sweet Country,"Warwick Thornton (director); David Tranter, Steven McGregor (screenplay...",Drama History Western,Warwick Thornton,Bryan Brown,Sam Neill,NaN
4,The Commuter,"Jaume Collet-Serra (director); Byron Willinger, Philip de Blasi (screen...",Action Thriller,Jaume Collet-Serra,Liam Neeson,Vera Farmiga,Patrick Wilson


In [85]:
movie = df_2018.loc[:,['director','actor1','actor2','actor3','genres','Title']]

In [86]:
movie = movie.rename(columns={'Title':'title'})

In [87]:
movie['actor2'] = movie['actor2'].replace(np.nan, 'unknown')
movie['actor3'] = movie['actor3'].replace(np.nan, 'unknown')

In [88]:
movie['combined'] = movie['actor1'] + ' ' + movie['actor2'] + ' '+ movie['actor3'] + ' '+ movie['director'] +' ' + movie['genres']

In [89]:
old_df = pd.read_csv('movie.csv')

In [90]:
final_df = old_df.append(movie,ignore_index=True)

In [91]:
final_df

,director,actor1,actor2,actor3,genres,title,combined
0,John Lasseter,Tom Hanks,Tim Allen,Don Rickles,Animation Comedy Family,Toy Story,Tom Hanks Tim Allen Don Rickles John Lasseter Animation Comedy Family
1,Joe Johnston,Robin Williams,Jonathan Hyde,Kirsten Dunst,Adventure Fantasy Family,Jumanji,Robin Williams Jonathan Hyde Kirsten Dunst Joe Johnston Adventure Fanta...
2,Howard Deutch,Walter Matthau,Jack Lemmon,Ann-Margret,Romance Comedy,Grumpier Old Men,Walter Matthau Jack Lemmon Ann-Margret Howard Deutch Romance Comedy
3,Forest Whitaker,Whitney Houston,Angela Bassett,Loretta Devine,Comedy Drama Romance,Waiting to Exhale,Whitney Houston Angela Bassett Loretta Devine Forest Whitaker Comedy Dr...
4,Charles Shyer,Steve Martin,Diane Keaton,Martin Short,Comedy,Father of the Bride Part II,Steve Martin Diane Keaton Martin Short Charles Shyer Comedy
...,...,...,...,...,...,...,...
37518,Garth Jennings,Matthew McConaughey,Reese Witherspoon,Scarlett Johansson,Animation Comedy Family Music,Sing 2,Matthew McConaughey Reese Witherspoon Scarlett Johansson Garth Jennings...
37519,Matthew Vaughn,Ralph Fiennes,Gemma Arterton,Rhys Ifans,Action Adventure Thriller War,The King's Man,Ralph Fiennes Gemma Arterton Rhys Ifans Matthew Vaughn Action Adventure...
37520,Joel Coen,Denzel Washington,Frances McDormand,Bertie Carvel,Drama War,The Tragedy of Macbeth,Denzel Washington Frances McDormand Bertie Carvel Joel Coen Drama War
37521,Denzel Washington,Michael B. Jordan,Chanté Adams,Jalon Christian,Drama Romance,A Journal for Jordan,Michael B. Jordan Chanté Adams Jalon Christian Denzel Washington Drama ...


In [92]:
final_df.isnull().sum()

director    0
actor1      0
actor2      0
actor3      0
genres      4
title       0
combined    4
dtype: int64

In [93]:
final_df.dropna(axis=0,how='any',inplace=True)

In [94]:
final_df.to_csv('complete_movie_data.csv',index=False)

In [95]:
#complete movie data is not suitable to deployment. Hence working with a smaller dataset

from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')
  

In [96]:
import pandas as pd
data = pd.read_csv("main_data.csv")
vector = cv.fit_transform(data['comb']).toarray()

In [97]:
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(vector)

In [98]:
def recommend(movie):
    index = data[data['movie_title'] == movie].index[0]
    distances = sorted(list(enumerate(similarity[index])),reverse=True,key = lambda x: x[1])
    for i in distances[1:6]:
        print(data.iloc[i[0]].movie_title)

In [99]:
recommend("batman")

batman returns
the legend of the lone ranger
raiders of the lost ark
the omega code
batman forever


In [100]:
#getting the movie id from tmdb
import numpy as np
def get_id(x):
    genres = []
    result = tmdb_movie.search(x)
    if result == []:
        movie_id = np.NaN
    else:
        movie_id = result[0].id
    return movie_id

In [101]:
data['movie_id'] = data['movie_title'].map(lambda x: get_id(str(x)))

In [102]:
data.isnull().sum()

director_name     0
actor_1_name      0
actor_2_name      0
actor_3_name      0
genres            0
movie_title       0
comb              0
movie_id         27
dtype: int64

In [103]:
data.dropna(axis=0, how='any',inplace=True)

In [104]:
data["movie_id"] = data["movie_id"].astype(float).astype(int)

In [105]:
data

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb,movie_id
0,James Cameron,CCH Pounder,Joel David Moore,Wes Studi,Action Adventure Fantasy Sci-Fi,avatar,CCH Pounder Joel David Moore Wes Studi James Cameron Action Adventure F...,19995
1,Gore Verbinski,Johnny Depp,Orlando Bloom,Jack Davenport,Action Adventure Fantasy,pirates of the caribbean: at world's end,Johnny Depp Orlando Bloom Jack Davenport Gore Verbinski Action Adventur...,285
2,Sam Mendes,Christoph Waltz,Rory Kinnear,Stephanie Sigman,Action Adventure Thriller,spectre,Christoph Waltz Rory Kinnear Stephanie Sigman Sam Mendes Action Adventu...,206647
3,Christopher Nolan,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,Action Thriller,the dark knight rises,Tom Hardy Christian Bale Joseph Gordon-Levitt Christopher Nolan Action ...,49026
4,Doug Walker,Doug Walker,Rob Walker,unknown,Documentary,star wars: episode vii - the force awakens,Doug Walker Rob Walker unknown Doug Walker Documentary,140607
...,...,...,...,...,...,...,...,...
6005,Joseph Kosinski,Tom Cruise,Miles Teller,Jennifer Connelly,Action Drama,top gun: maverick,Tom Cruise Miles Teller Jennifer Connelly Joseph Kosinski Action Drama,361743
6006,Joel Crawford,Nicolas Cage,Emma Stone,Ryan Reynolds,Animation Adventure Family,the croods 2,Nicolas Cage Emma Stone Ryan Reynolds Joel Crawford Animation Adventure...,529203
6007,Liesl Tommy,Jennifer Hudson,Forest Whitaker,Marlon Wayans,Music Drama,respect,Jennifer Hudson Forest Whitaker Marlon Wayans Liesl Tommy Music Drama,592863
6008,Ridley Scott,Matt Damon,Adam Driver,Jodie Comer,Drama,the last duel,Matt Damon Adam Driver Jodie Comer Ridley Scott Drama,617653


In [106]:
data.to_csv('new_movie_data.csv')

In [107]:
import pickle
pickle.dump(data,open('movie_list.pkl','wb'))
pickle.dump(similarity,open('similarity.pkl','wb'))